# Importing Libraries

In [28]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer

#nltk.download('all')

# Reading Docs

In [29]:
path = "corpus/"

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        files.append(os.path.join(r, file))

# Removing Headers and HTML Tags, Tokenization, Lower Casing

In [30]:
docs_words = {}
i = 0 #for docs_words

for f in files:
    doc = open(f, errors='ignore')
    html = doc.read()
    
    index = html.find("<html")
    if index == -1 :
        html.find("<Html")
    elif index == -1 : 
        html.find("<HTML")
    html = html[index:]

    soup = BeautifulSoup(html)

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    
    #print("Doc : " + text + "\n\n")
    tknzr = TweetTokenizer()
    tokens = tknzr.tokenize(text)
    words_pos = {}
    j = 0; #for words_pos
    for t in tokens:
        t = t.lower() #lowering Case
        
        if t not in words_pos.keys():
            words_pos[t] = []
        if t in words_pos.keys():
            words_pos[t].append(j)
        j = j + 1
    
    docs_words[i] = words_pos
    i = i + 1

#print(docs_words) 

# Removing stop words

In [31]:
f = open("stoplist.txt", "r") 
stoplist = f.read().splitlines() #Stoplist words

for i in range(len(docs_words)):
    for key in list(docs_words[i]):
        if key in stoplist or len(key) == 1:
            docs_words[i].pop(key,None)
#print(docs_words)            

# Stemming 

In [32]:
ps = PorterStemmer()
for i in range(len(docs_words)):
    for key in list(docs_words[i]):
        stem = ps.stem(key)
        stem = stem.replace("'", "")
        if stem != key:
            if stem not in docs_words[i].keys():
                docs_words[i][stem] = docs_words[i].pop(key)
            elif stem in docs_words[i].keys():
                docs_words[i][stem] = docs_words[i][stem] + docs_words[i].pop(key)
            
#print(docs_words)        

# Inverted Index using Hash Map

In [33]:
words_docs = {}

for i in range(len(docs_words)):
    for key in docs_words[i].keys():
        docs_pos = {}
        docs_pos[i] = docs_words[i].get(key)
        if key not in words_docs.keys():
            words_docs[key] = docs_pos
        elif key in words_docs.keys():
             words_docs[key][i] = docs_pos[i]

#print(words_docs)        

# Writing docids.txt 

In [34]:
f_docs = open("docids.txt", 'w')
i = 0
for f in files:
    doc_name = os.path.basename(f)
    f_docs.write(str(i) + "\t" + doc_name + "\n")
    i = i + 1

# Writing termids.txt

In [35]:
f_terms = open("termids.txt", 'w', errors="ignore")
id = 0

for key in words_docs.keys():
    f_terms.write(str(id) + "\t" + key + "\n")
    id = id + 1

In [41]:
print(words_docs["apple"])

{1060: [685], 1794: [130, 258], 2056: [1131], 2175: [940], 2176: [940], 2177: [940], 2780: [541, 632], 3328: [1143], 3426: [7026, 16902]}
